---


<div style="text-align: center;">
    <h1>Variance Analysis Project - Fırat Olçum - 20.01.2025</h1>
</div>
<div style="text-align: center;"> 


</div>

---

# 📑 İçindekiler

1. [Varyans Analizi Tanımı](#section-1)
2. [Bağımlı ve Bağımsız Değişken Seçimi](#section-2)
3. [Varyans Analizine Uygunluk Testi](#section-3)
   - [Grupların Birbirinden Bağımsız Olması](#section-10)
   - [Normallik Testi](#section-11)
   - [Logaritmik Veri Dönüşümü](#section-12)
   - [Varyansların Homojenliği Testi](#section-13)
5. [Modelin Kurulması](#section-4)
6. [Varyans Analizi Yapılması](#section-5)
7. [Sonuçların Yorumlanması](#section-7)
8. [Post-Hoc Testleri](#section-8)
   - [Tukey HSD Testi](#section-14)
   - [Bonferroni Testi](#section-15)
8. [Genel Değerlendirme](#section-9)

## <a id="section-1"></a>  1. Varyans Analizi Tanımı

[🔝 İçindekilere Dön](#📑-İçindekiler)

Varyans Analizi (veya ANOVA) istatistik bilim dalında, grup ortalamaları ve (gruplar içi ve gruplar arası varyasyon gibi) bunlara bağlı olan işlemleri analiz etmek için kullanılan bir istatistiksel modeller koleksiyonudur. 

Varyans Analizi kullanılmaktayken belirlenmiş bir değişkenin gözlemlenen varyansı farklı değişim kaynaklarına dayandırılabilen varyans bileşenine ayrılır. 

En basit şekliyle varyans analizi birkaç grubun ortalamalarının birbirine eşit mi eşit değil mi olduğunu sınamak için bir çıkarımsal istatistik sınaması olur ve bu sınama iki-grup için yapılan t-test sınamasını çoklu-gruplar için genelleştirir.

## <a id="section-2"></a> 2. Bağımlı ve Bağımsız Değişken Seçimi
[🔝 İçindekilere Dön](#📑-İçindekiler)

**Bağımlı değişken:** Modelde açıklamak istediğimiz değişkendir ("ERP" değerleri).

**Bağımsız değişken:** Bağımlı değişkeni etkileyebileceğini düşündüğümüz faktördür ("VendorName").

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("computer+hardware.csv", index_col = 0)

In [2]:
df = df.drop(columns = ["ModelName", "PRP"])

## <a id="section-3"></a> 3. Varyans Analizine Uygunluk Testi

[🔝 İçindekilere Dön](#📑-İçindekiler)


Varyans analizi yapmadan önce, veri setindeki varsayımları kontrol etmemiz gerekiyor:
- Grupların Birbirinden Bağımsız Olması
- Normallik Testi (Shapiro-Wilk Testi)
- Varyansların Homojenliği Testi (Levene Testi)
  
Bu testler, varyans analizine uygun olup olmadığınızı belirleyecektir.

### <a id="section-10"></a> 3.1 Grupların Birbirinden Bağımsız Olması

[🔝 İçindekilere Dön](#📑-İçindekiler)

Kullandığımız veri setinde marka isimleri birbirinden bağımsızdır.

### <a id="section-11"></a> 3.2 Normallik Testi (Shapiro-Wilk)

[🔝 İçindekilere Dön](#📑-İçindekiler)

In [3]:
import scipy.stats as stats

In [4]:
def normallik_testi(grup_verisi):
    W, p_degeri = stats.shapiro(grup_verisi)
    return p_degeri

In [5]:
df["VendorName"].value_counts(sort=True)[:6]
# En çok veri içeren 6 vendor üzerinden çalışma yapacağım.

VendorName
ibm          32
nas          19
ncr          13
sperry       13
honeywell    13
siemens      12
Name: count, dtype: int64

In [6]:
# Normallik testi her grup için
grup_adlari = ["ibm", "nas", "ncr", "sperry", "honeywell", "siemens"]
normallik_sonuclari = {}

for grup in grup_adlari:
    grup_verisi = df[df["VendorName"] == grup]["ERP"]
    p_degeri = normallik_testi(grup_verisi)
    normallik_sonuclari[grup] = p_degeri

# Normallik testinin sonuçları

print("Shapiro-Wilk Testi Sonuçları (Normallik Testi):")
for grup, p_degeri in normallik_sonuclari.items():
    print(f"{grup}: p-değeri = {p_degeri:.4f}")

Shapiro-Wilk Testi Sonuçları (Normallik Testi):
ibm: p-değeri = 0.0000
nas: p-değeri = 0.0005
ncr: p-değeri = 0.0064
sperry: p-değeri = 0.0007
honeywell: p-değeri = 0.0004
siemens: p-değeri = 0.0168


**P-değeri 0.05' ten küçükse, verinin normal dağılıma uymadığı kabul edilir.**

**Sonuçları incelediğimizde tüm gruplar için de veriler normal dağılıma uymuyor gibi görünüyor.**

**Normal dağılım ön şartını sağlamak için logaritmik veri dönüşümü yapacağım.**

### <a id="section-12"></a> 3.3 Logaritmik Veri Dönüşümü

[🔝 İçindekilere Dön](#📑-İçindekiler)

In [7]:
df['ERP_log'] = np.log(df['ERP'])

# Shapiro-Wilk testini yeniden uygulama
from scipy.stats import shapiro

# Testi her grup için uygulayalım
grup1_log = df[df['VendorName'] == 'ibm']['ERP_log']
grup2_log = df[df['VendorName'] == 'nas']['ERP_log']
grup3_log = df[df['VendorName'] == 'ncr']['ERP_log']
grup4_log = df[df['VendorName'] == 'sperry']['ERP_log']
grup5_log = df[df['VendorName'] == 'honeywell']['ERP_log']
grup6_log = df[df['VendorName'] == 'siemens']['ERP_log']

stat_ibm_log, p_ibm_log = shapiro(grup1_log)
stat_nas_log, p_nas_log = shapiro(grup2_log)
stat_ncr_log, p_ncr_log = shapiro(grup3_log)
stat_sperry_log, p_sperry_log = shapiro(grup4_log)
stat_honeywell_log, p_honeywell_log = shapiro(grup5_log)
stat_siemens_log, p_siemens_log = shapiro(grup6_log)

print("Log Dönüşümü Sonrası Shapiro-Wilk Testi Sonuçları:")
print(f"IBM: p-değeri = {p_ibm_log}")
print(f"NAS: p-değeri = {p_nas_log}")
print(f"NCR: p-değeri = {p_ncr_log}")
print(f"sperry: p-değeri = {p_sperry_log}")
print(f"honeywell: p-değeri = {p_honeywell_log}")
print(f"siemens: p-değeri = {p_siemens_log}")

Log Dönüşümü Sonrası Shapiro-Wilk Testi Sonuçları:
IBM: p-değeri = 0.003561823396012187
NAS: p-değeri = 0.5607991218566895
NCR: p-değeri = 0.7909696102142334
sperry: p-değeri = 0.042860373854637146
honeywell: p-değeri = 0.008238082751631737
siemens: p-değeri = 0.8759718537330627


Log dönüşümü sonrasında yapılan Shapiro-Wilk testine göre:

- **IBM:** p-değeri = 0.00356 (0.05'ten küçük), bu da verinin normal dağılımdan sapma gösterdiği anlamına gelir.
- **NAS:** p-değeri = 0.5608 (0.05'ten büyük), bu da verinin normal dağılıma **uygun** olduğunu gösterir.
- **NCR:** p-değeri = 0.7910 (0.05'ten büyük), bu da verinin normal dağılıma **uygun** olduğunu gösterir.
- **Sperry:** p-değeri = 0.0429 (0.05'ten küçük), bu da verinin normal dağılımdan sapma gösterdiği anlamına gelir.
- **Honeywell:** p-değeri = 0.0082 (0.05'ten küçük), bu da verinin normal dağılımdan sapma gösterdiği anlamına gelir.
- **Siemens:** p-değeri = 0.7910 (0.05'ten büyük), bu da verinin normal dağılıma **uygun** olduğunu gösterir.

### <a id="section-13"></a> 3.4 Varyansların Homojenliği Testi (Levene Testi)

[🔝 İçindekilere Dön](#📑-İçindekiler)

In [8]:
grup_adlari = ["nas", "ncr", "siemens"]

def varyans_homojenligi_testi():
    gruplar = [df[df['VendorName'] == grup]['ERP_log'] for grup in grup_adlari]
    stat, p_degeri = stats.levene(*gruplar)
    return p_degeri

# Levene testi sonucu
p_degeri_levene = varyans_homojenligi_testi()
print("\nLevene Testi Sonucu (Varyansların Homojenliği Testi):")
print(f"Levene Testi p-değeri: {p_degeri_levene:.4f}")


Levene Testi Sonucu (Varyansların Homojenliği Testi):
Levene Testi p-değeri: 0.6995


Bu sonuç, p > 0.05 olduğu için varyansların homojen olduğu sonucuna varılabilir. Yani, veri setindeki grupların varyansları birbirine benzer ve varyanslar homojen kabul edilebilir. Bu, varyans analizi (ANOVA) için geçerli bir varsayımdır.

## <a id="section-4"></a> 4. Modelin Kurulması
[🔝 İçindekilere Dön](#📑-İçindekiler)

Varyans analizini (ANOVA) uygulayacağımız modeli kuracağız. Modelde bağımlı değişken "ERP_log", bağımsız değişken ise "VendorName" olacak.

In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [10]:
# Modeli kuruyoruz
model = ols('ERP_log ~ C(VendorName)', 
        data=df).fit()

## <a id="section-5"></a> 5. Varyans Analizi Yapılması
[🔝 İçindekilere Dön](#📑-İçindekiler)

Varyans analizini yaparak, bağımsız değişkenin bağımlı değişken üzerindeki etkisini test edeceğiz.

In [11]:
from statsmodels.stats.anova import anova_lm

# ANOVA analizini yapıyoruz
anova_sonuc = anova_lm(model)
print("\nVaryans Analizi Sonuçları:")
print(anova_sonuc)


Varyans Analizi Sonuçları:
                  df      sum_sq   mean_sq        F        PR(>F)
C(VendorName)   29.0   73.041565  2.518675  4.11651  2.055708e-09
Residual       179.0  109.520625  0.611847      NaN           NaN


**F-değeri:** 4.11651, bağımsız değişkenin (VendorName) varyans üzerindeki etkisini değerlendiren testin sonucudur.

**p-değeri:** 2.055708e-09, çok küçük bir değerdir ve p < 0.05 olduğundan, gruplar arasında istatistiksel olarak anlamlı bir fark olduğu sonucuna varılır.

## <a id="section-7"></a> 6. Sonuçların Yorumlanması
[🔝 İçindekilere Dön](#📑-İçindekiler)

ANOVA sonucuna göre p-değerini inceleyerek, bağımsız değişkenin bağımlı değişken üzerinde etkili olup olmadığını yorumlayacağız.

In [12]:
# p-değerini inceleyerek anlamlılık durumunu yorumlayalım
p_degeri = anova_sonuc['PR(>F)'][0]
if p_degeri < 0.05:
    print("VendorName, ERP üzerinde anlamlı bir etkiye sahiptir.")
else:
    print("VendorName, ERP üzerinde anlamlı bir etkiye sahip değildir.")


VendorName, ERP üzerinde anlamlı bir etkiye sahiptir.


## <a id="section-8"></a> 7. Post-Hoc Testleri
[🔝 İçindekilere Dön](#📑-İçindekiler)

ANOVA testinden anlamlı bir fark bulduğumuz için, bu farkın hangi gruplar arasında olduğunu görmek için post-hoc testleri yapacağız.

 ### <a id="section-14"></a> 7.1 Tukey HSD Testi
[🔝 İçindekilere Dön](#📑-İçindekiler)

In [13]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
df = df[df["VendorName"].isin(grup_adlari)].reset_index(drop=True)
# Tukey HSD testi
tukey_test = pairwise_tukeyhsd(df['ERP_log'], df['VendorName'], alpha=0.05)

print("\nTukey HSD Testi Sonuçları:")
print(tukey_test.summary())



Tukey HSD Testi Sonuçları:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1  group2 meandiff p-adj   lower  upper  reject
----------------------------------------------------
   nas     ncr  -0.6149 0.1366   -1.38 0.1502  False
   nas siemens  -0.3933 0.4483  -1.177 0.3905  False
   ncr siemens   0.2216 0.8027 -0.6293 1.0726  False
----------------------------------------------------


**Tukey HSD testine göre gruplar arasındaki hiçbir karşılaştırmanın anlamlı bir fark yaratmadığı görünüyor.**

### <a id="section-15"></a> 7.2 Bonferroni Testi
[🔝 İçindekilere Dön](#📑-İçindekiler)

In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

# Varyans analizi sonucu (ANOVA) yapılmış veri üzerinden çalışacağız
# İlgili modelin oluşturulması
model = ols('ERP_log ~ C(VendorName)', data=df).fit()

# Varyans analizini yapma
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# Çoklu karşılaştırma için Bonferroni testi
mc = MultiComparison(df['ERP_log'], df['VendorName'])

# Bonferroni testi uygulama
result = mc.allpairtest(stats.ttest_ind, method='bonferroni')

# Sonuçları yazdırma
print(result[0])


                  sum_sq    df         F    PR(>F)
C(VendorName)   3.098203   2.0  2.027363  0.144674
Residual       31.327964  41.0       NaN       NaN
Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonferroni
alphacSidak=0.02, alphacBonf=0.017
group1  group2   stat   pval  pval_corr reject
----------------------------------------------
   nas     ncr  2.0479 0.0494    0.1482  False
   nas siemens   1.197  0.241     0.723  False
   ncr siemens -0.6131 0.5458       1.0  False
----------------------------------------------


**Bonferroni testi ile yapılan çoklu karşılaştırmalar sonucunda da herhangi bir çift arasında anlamlı fark bulunmamıştır.**

---

## <a id="section-9"></a> 8. Genel Değerlendirme
[🔝 İçindekilere Dön](#📑-İçindekiler)

**Seçtiğimiz VendorName faktörüne göre ERP_log üzerinde istatistiksel olarak anlamlı bir fark gözlemlenmemektedir.**

**Bu, ERP_log değerlerinin vendorlar arasında önemli ölçüde farklılaşmadığı anlamına gelir.**

**Yani, VendorName (nas, ncr, siemens) gruplarının ERP_log üzerindeki etkileri istatistiksel olarak anlamlı bulunmamıştır.**

---


<div style="text-align: center;"> 


</div>

<div style="text-align: center;">
    <h1>Variance Analysis Project - Fırat Olçum - 20.01.2025</h1>
</div>

---